In [1]:
import numpy as np 
import torch
import random
import os

# customized 
from psml_defense import PsmlDefenseProxy

In [2]:
# CIFAR 
pareto_front_models = [
    'shufflenetv2_x0_5',
    'resnet20',
    'mobilenetv2_x0_5',
    'vgg11_bn',
    'shufflenetv2_x2_0',
    'resnet32',
    'shufflenetv2_x1_5',
    'vgg13_bn',
    'shufflenetv2_x1_0',
    'resnet44',
    'vgg16_bn',
    'mobilenetv2_x1_0',
    'mobilenetv2_x0_75',
    'mobilenetv2_x1_4',
    'resnet56',
    'vgg19_bn',
    'repvgg_a0',
    'repvgg_a2',
    'repvgg_a1',
    'vit_small_patch16_384',
    'convnext_tiny',
    'vit_base_patch16_384',
    'vit_large_patch16_384',
    'convnext_base',
    'convnext_large'
]

pareto_front_spec = [
    [90.6, 11.07],
    [92.68, 12.83],
    [93.14, 15.14],
    [93.22, 17.45],
    [93.68, 18.17],
    [93.7, 26.64],
    [93.7, 33.15],
    [93.72, 35.86],
    [93.74, 36.27],
    [93.92, 36.74],
    [93.92, 37.05],
    [93.94, 37.24],
    [94.08, 37.28],
    [94.24, 40.81],
    [94.26, 41.06],
    [94.3, 42.77],
    [94.32, 43.98],
    [95.22, 44.26],
    [95.5, 44.63],
    [98.02, 45.31],
    [98.06, 46.28],
    [98.32, 47.42],
    [98.38, 67.79],
    [98.62, 71.79],
    [98.96, 77.57],
]

psml_pareto_front = np.array(pareto_front_spec)

In [3]:
eps = 100

In [4]:
defense = PsmlDefenseProxy(pareto_front_models=pareto_front_models, pareto_front_spec=pareto_front_spec, eps=eps, sensitivity=1)

Initializing PsmlDefenseProxy... w/ eps 100, sensitivity 1


In [5]:
defense.pareto_front

array([[90.6 , 11.07],
       [92.68, 12.83],
       [93.14, 15.14],
       [93.22, 17.45],
       [93.68, 18.17],
       [93.7 , 26.64],
       [93.7 , 33.15],
       [93.72, 35.86],
       [93.74, 36.27],
       [93.92, 36.74],
       [93.92, 37.05],
       [93.94, 37.24],
       [94.08, 37.28],
       [94.24, 40.81],
       [94.26, 41.06],
       [94.3 , 42.77],
       [94.32, 43.98],
       [95.22, 44.26],
       [95.5 , 44.63],
       [98.02, 45.31],
       [98.06, 46.28],
       [98.32, 47.42],
       [98.38, 67.79],
       [98.62, 71.79],
       [98.96, 77.57]])

In [6]:
defense.pareto_front_map

{(90.6, 11.07): 'shufflenetv2_x0_5',
 (92.68, 12.83): 'resnet20',
 (93.14, 15.14): 'mobilenetv2_x0_5',
 (93.22, 17.45): 'vgg11_bn',
 (93.68, 18.17): 'shufflenetv2_x2_0',
 (93.7, 26.64): 'resnet32',
 (93.7, 33.15): 'shufflenetv2_x1_5',
 (93.72, 35.86): 'vgg13_bn',
 (93.74, 36.27): 'shufflenetv2_x1_0',
 (93.92, 36.74): 'resnet44',
 (93.92, 37.05): 'vgg16_bn',
 (93.94, 37.24): 'mobilenetv2_x1_0',
 (94.08, 37.28): 'mobilenetv2_x0_75',
 (94.24, 40.81): 'mobilenetv2_x1_4',
 (94.26, 41.06): 'resnet56',
 (94.3, 42.77): 'vgg19_bn',
 (94.32, 43.98): 'repvgg_a0',
 (95.22, 44.26): 'repvgg_a2',
 (95.5, 44.63): 'repvgg_a1',
 (98.02, 45.31): 'vit_small_patch16_384',
 (98.06, 46.28): 'convnext_tiny',
 (98.32, 47.42): 'vit_base_patch16_384',
 (98.38, 67.79): 'vit_large_patch16_384',
 (98.62, 71.79): 'convnext_base',
 (98.96, 77.57): 'convnext_large'}

In [12]:
#all_epsilons = [1,2,3,4,5,10,50,100,500,1000]
all_epsilons = [0.01, 0.05, 0.1, 0.2, 0.5, 1,5,10, 20, 30, 50,100,200]

In [21]:
# queries = [(random.randint(50, 100), random.randint(10, 250)) for _ in range(20)]
# queries = [(72.18, 18.5) for _ in range(20)] # vgg19
queries = [(94.26, 41.06) for _ in range(4000)] # resnet56
sensitivity = 0.01

In [22]:
from collections import Counter

for eps in all_epsilons: 
    server = PsmlDefenseProxy(pareto_front_models=pareto_front_models, pareto_front_spec=pareto_front_spec, eps=eps, sensitivity=sensitivity)
    
    print(f"\ncomputing goodput for eps={eps}...")
    utility = 0
    answered_queries = 0
    selected_points = [] 
    selected_models = [] 
    
    for query in queries:
        # selected_element: (accuracy, latency)
        selected_element = server.l1_permute_and_flip_mechanism(query)
        # we can get none when the query is infeasible, then no point will have a high enough prob to be selected
        # print(query, ' -> ', selected_element)
        if selected_element is not None:
            answered_queries += 1
            selected_points.append(selected_element)
            selected_models.append(server.m_query(selected_element))
            
            utility += server.l1_score(float(selected_element[0]), float(selected_element[1]), query[0], query[1])
    
    print('utility score: ', utility / answered_queries)
    # for point, model in zip(selected_points, selected_models):
    #    print(point, ' -> ', model.__class__.__name__)
    
    model_counts = Counter([model for model in selected_models])
    total_models = sum(model_counts.values())  # Total selections
    
    # Calculate and display proportions
    sorted_model_counts = {model: model_counts.get(model, 0) for model in pareto_front_models}
    
    # Calculate and display proportions
    print("model selection:")
    for model_name in pareto_front_models:
        count = sorted_model_counts.get(model_name, 0)
        proportion = count / total_models if total_models > 0 else 0
        print(f"{model_name}: {count} ({proportion:.2%})")

Initializing PsmlDefenseProxy... w/ eps 0.01, sensitivity 0.01

computing goodput for eps=0.01...
utility score:  0.9751836749999836
model selection:
shufflenetv2_x0_5: 152 (3.80%)
resnet20: 159 (3.98%)
mobilenetv2_x0_5: 137 (3.43%)
vgg11_bn: 173 (4.32%)
shufflenetv2_x2_0: 156 (3.90%)
resnet32: 138 (3.45%)
shufflenetv2_x1_5: 150 (3.75%)
vgg13_bn: 166 (4.15%)
shufflenetv2_x1_0: 169 (4.23%)
resnet44: 159 (3.98%)
vgg16_bn: 155 (3.88%)
mobilenetv2_x1_0: 221 (5.53%)
mobilenetv2_x0_75: 168 (4.20%)
mobilenetv2_x1_4: 151 (3.77%)
resnet56: 175 (4.38%)
vgg19_bn: 166 (4.15%)
repvgg_a0: 166 (4.15%)
repvgg_a2: 180 (4.50%)
repvgg_a1: 140 (3.50%)
vit_small_patch16_384: 170 (4.25%)
convnext_tiny: 167 (4.17%)
vit_base_patch16_384: 142 (3.55%)
vit_large_patch16_384: 150 (3.75%)
convnext_base: 150 (3.75%)
convnext_large: 140 (3.50%)
Initializing PsmlDefenseProxy... w/ eps 0.05, sensitivity 0.01

computing goodput for eps=0.05...
utility score:  0.9784353999999839
model selection:
shufflenetv2_x0_5: 150 (